# Variants

In [ ]:
from bitarray import bitarray
import Bio.Seq

class CoreBitMask:
    
    def __init__(self, sequence: Bio.Seq.Seq = None, existing_bitmask: bitarray = None):
        if existing_bitmask is not None and sequence is not None:
            raise Exception(f'Cannot set both existing_bitmask={existing_bitmask} and sequence={sequence}')
            
        if existing_bitmask:
            self._core_bitmask = existing_bitmask
        elif sequence:
            self._core_bitmask = bitarray(len(sequence))
            self._core_bitmask.setall(True)
            self._add_sequence(sequence)
        else:
            raise Exception('If no existing_bitmask set then sequence must be defined')
            
    def _add_sequence(self, sequence: Bio.Seq.Seq) -> float:
        for idx, char in enumerate(sequence):
            if char.upper() == 'N' or char == '-':
                self._core_bitmask[idx] = False
                
    def get_bytes(self):
        return self._core_bitmask.tobytes()
    
    def core_length(self) -> int:
        return self._core_bitmask.count()
    
    def core_proportion(self) -> float:
        return self.core_length()/len(self)
    
    def __len__(self) -> int:
        return len(self._core_bitmask)
    
    def __getitem__(self, index: int) -> bool:
        return self._core_bitmask[index]
    
s = Bio.Seq.Seq('ATCG-NN')
mask = CoreBitMask(sequence=s)
mask.core_proportion()

In [ ]:
from typing import List, Dict
import vcf
import Bio.Seq
from Bio import SeqIO
import pandas as pd
import re
import os

class VariantsReader:
    
    def __init__(self):
        pass

    def read_vcf(self, file: str) -> pd.DataFrame:
        reader = vcf.Reader(open(file, 'r'))
        df = pd.DataFrame([vars(r) for r in reader])
        out = df.merge(pd.DataFrame(df.INFO.tolist()),
                       left_index=True, right_index=True)
        out = out[['CHROM', 'POS', 'REF', 'ALT', 'DP', 'QUAL', 'RO', 'AO', 'INFO']]
        out['TYPE'] = out['INFO'].map(lambda x: x['TYPE'][0])
        out = out.drop('INFO', axis='columns')
        out['ALT'] = out['ALT'].map(lambda x: str(x[0]))
        out['REF'] = out['REF'].map(lambda x: str(x[0]))
        out['AO'] = out['AO'].map(lambda x: x[0])
        cols = out.columns.tolist()
        out['FILE'] = os.path.basename(file)
        out = out.reindex(columns=['FILE'] + cols)
        return out

    def read_vcfs(self, files: List[str]) -> pd.DataFrame:
        frames = [self.read_vcf(f) for f in files]
        return pd.concat(frames)
    
    def read_core_masks(self, bit_mask_files_dir: str) -> Dict[str, Dict[str, CoreBitMask]]:
        files = [os.path.join(bit_mask_files_dir, f) for f in os.listdir(bit_mask_files_dir) if f.endswith('.fa')]
        core_masks = {}
        for file in files:
            name = re.sub('\.aligned\.fa$', '', os.path.basename(file))
            core_masks[name] = {}
            for record in SeqIO.parse(file, 'fasta'):
                if record.id not in core_masks[name]:
                    core_masks[name][record.id] = CoreBitMask(sequence=record.seq)
                    print(f'{name}:{record.id} -> {core_masks[name][record.id].core_proportion()}')
                    
        return core_masks

directory = 'data/snps-vcf'
files = [os.path.join(directory, f) for f in os.listdir('data/snps-vcf')]
files = [f for f in files if f.endswith('.vcf')]

vr = VariantsReader()
core_masks = vr.read_core_masks('data/snps-aligned')
df = vr.read_vcfs(files)
df

In [ ]:
import re

sample_names = df.FILE.value_counts().index.tolist()
sample_names = [re.sub('\.filt\.vcf$', '', n) for n in sample_names]
sample_names

# DB model

In [ ]:
from typing import List, Any

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

from bitarray import bitarray

Base = declarative_base()
Base

from sqlalchemy import Column, Integer, String, Sequence, BigInteger, ForeignKey, Table, LargeBinary
    
association_table = Table('sample_variation_allele', Base.metadata,
    Column('sample_id', Integer, ForeignKey('sample.id')),
    Column('variantion_allele_id', String, ForeignKey('variation_allele.id')),
)

class VariationAllele(Base):
    __tablename__ = 'variation_allele'
    id = Column(String, primary_key=True)
    sequence_id = Column(String, ForeignKey('reference_sequence.id'))
    position = Column(Integer, primary_key=True)
    ref = Column(String(255), primary_key=True)
    alt = Column(String(255), primary_key=True)
    var_type = Column(String(255))
    
    samples = relationship('Sample', secondary=association_table, back_populates='variants')
    sequence = relationship('ReferenceSequence', back_populates='variants')
    
    def __init__(self, sequence = None, position: int = -1, ref: str = None, alt: str = None,
                 var_type: str = None):
        self.sequence = sequence
        self.position = position
        self.ref = ref
        self.alt = alt
        self.var_type = var_type
        
        self.id = self.to_spdi()
    
    def to_spdi(self):
        return f'{self.sequence.sequence_name}:{self.position}:{self.ref}:{self.alt}'
    
    def __repr__(self):
        return (f'<VariationAllele(sequence_name={self.sequence.sequence_name}'
                f', position={self.position}, ref={self.ref}, alt={self.alt}, var_type={self.var_type})>')

class Reference(Base):
    __tablename__ = 'reference'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    length = Column(Integer)
    sequences = relationship('ReferenceSequence')
    
    def __repr__(self):
        return f'<Reference(id={self.id}, name={self.name}, length={self.length})>'
    
    
class SampleSequence(Base):
    __tablename__ = 'sample_sequence'
    sample_id = Column(Integer, ForeignKey('sample.id'), primary_key=True)
    sequence_id = Column(Integer, ForeignKey('reference_sequence.id'), primary_key=True)
    core_mask = Column(LargeBinary)
    flag = Column(String(255))
    
    sequence = relationship('ReferenceSequence', back_populates='sample_sequences')
    sample = relationship('Sample', back_populates='sample_sequences')
    
    def get_core_mask(self):
        if self.core_mask is None:
            raise Exception('core_mask is not set')
        else:
            barray = bitarray()
            barray.frombytes(self.core_mask)
            
            # Since I'm decoding a bitarray from bytes, the bytes must always be a multiple of 8
            # But the sequence length can be a non-multiple of 8
            # So I have to remove (slice) the few additional elements from this array that got added
            # When decoding from bytes
            barray = barray[:self.sequence.sequence_length]
            return CoreBitMask(existing_bitmask=barray)
        
    def set_core_mask(self, core_mask: CoreBitMask) -> None:
        if core_mask is None:
            raise Exception('Cannot set core_mask to None')
        else:
            self.core_mask = core_mask.get_bytes()
    
    def __repr__(self):
        return f'<SampleSequence(sample_id={self.sample_id}, sequence_id={self.sequence_id}, flag={self.flag})>'
    
    
class ReferenceSequence(Base):
    __tablename__ = 'reference_sequence'
    id = Column(Integer, primary_key=True)
    reference_id = Column(Integer, ForeignKey('reference.id'))
    sequence_name = Column(String(255))
    sequence_length = Column(Integer)
    
    variants = relationship('VariationAllele', back_populates='sequence')
    sample_sequences = relationship('SampleSequence', back_populates='sequence')
    
    def __repr__(self):
        return (f'<ReferenceSequence(id={self.id}, sequence_name={self.sequence_name},'
                f'sequence_length={self.sequence_length}, reference_id={self.reference_id})>')
    
    
class Sample(Base):
    __tablename__ = 'sample'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    
    variants = relationship('VariationAllele', secondary=association_table, back_populates='samples')
    sample_sequences = relationship('SampleSequence', back_populates='sample')
    
    def __repr__(self):
        return f'<Sample(id={self.id}, name={self.name})>'

# Create some data

In [ ]:
from Bio import SeqIO

ref_name = '2011C-3609.fasta'
ref_length = 0
ref_contigs = {}
for record in SeqIO.parse(f"reference/{ref_name}", "fasta"):
    ref_contigs[record.id] = ReferenceSequence(sequence_name=record.id, sequence_length=len(record.seq))
    ref_length += len(record.seq)

reference = Reference(name = ref_name, length = ref_length, sequences=list(ref_contigs.values()))
ref_contigs

In [ ]:
from typing import Dict, List
import logging
import pandas as pd
import re

logger = logging.getLogger('VariationService')
logger.setLevel(logging.DEBUG)
#logging.basicConfig(level=logging.DEBUG)

class VariationService:
    
    def __init__(self, session):
        self._session = session
        
    def _create_file_variants(self, var_df: pd.DataFrame, ref_contigs: Dict[str, ReferenceSequence]) -> Dict[str, List[VariationAllele]]:
        variant_table = {}
        file_variants = {}
        sample_sequences = {}
        for row in var_df.iterrows():
            sample_name = re.sub('\.filt\.vcf$', '', row[1]['FILE'])

            ref_contig = ref_contigs[row[1]['CHROM']]
            variant = VariationAllele(sequence=ref_contig, position=row[1]['POS'],
                                     ref=row[1]['REF'], alt=row[1]['ALT'], var_type=row[1]['TYPE'])
            if variant.id not in variant_table:
                variant_table[variant.id] = variant
            else:    
                variant = variant_table[variant.id]

            if sample_name not in file_variants:
                file_variants[sample_name] = []

            file_variants[sample_name].append(variant)

        return file_variants
    
    def insert_variants(self, var_df: pd.DataFrame, ref_contigs: Dict[str, ReferenceSequence],
                       core_masks: Dict[str, Dict[str, CoreBitMask]]) -> None:
        file_variants = self._create_file_variants(var_df, ref_contigs)
        
        for s in file_variants:
            ref_objects = {ref_contigs[v.sequence.sequence_name] for v in file_variants[s]}
            sample_core_masks = core_masks[s]
            sample_sequences = []
            for r in ref_objects:
                sample_sequence = SampleSequence(sequence=r)
                sample_sequence.set_core_mask(sample_core_masks[r.sequence_name])
                sample_sequences.append(sample_sequence)
            sample = Sample(name=s, variants=file_variants[s], sample_sequences=sample_sequences)
            self._session.add(sample)
            
        self._session.commit()
        
    def pairwise_distance(self, samples: List[str], var_type = 'all', distance_type = 'jaccard') -> pd.DataFrame:
        sample_objs = self._session.query(Sample).filter(Sample.name.in_(samples)).all()
        
        if var_type == 'all':
            sample_variants = {s.name: {v.to_spdi() for v in s.variants} for s in sample_objs}
        else:
            sample_variants = {s.name: {v.to_spdi() for v in s.variants if v.var_type == var_type} for s in sample_objs}
        
        names = sample_variants.keys()
        distances = []
        for name1 in names:
            row = []
            for name2 in names:
                if name1 == name2:
                    row.append(0)
                else:
                    if distance_type == 'jaccard':
                        logger.debug(f'variants1=[{sample_variants[name1]}]')
                        logger.debug(f'variants2=[{sample_variants[name2]}]')
                        intersection = sample_variants[name1].intersection(sample_variants[name2])
                        union = sample_variants[name1].union(sample_variants[name2])
                        
#                         print(f'unique_variants1=[{sample_variants[name1] - sample_variants[name2]}]')
#                         print(f'unique_variants2=[{sample_variants[name2] - sample_variants[name1]}]')
                        
                        row.append(1 - (len(intersection)/len(union)))
                    else:
                        raise Exception(f'Unsupported distance_type=[{distance_type}]')
            distances.append(row)
            
        return pd.DataFrame(distances, columns=names, index=names)

# Insert into database

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///:memory:', echo=True)

Session = sessionmaker(bind=engine)
session = Session()
session

Base.metadata.create_all(engine)

In [ ]:
session.add(reference)
session.commit()

In [ ]:
ref = session.query(Reference).filter_by(id = 1).first()
ref_contigs = {c.sequence_name: c for c in ref.sequences}
ref_contigs

In [ ]:
dfs = df[(df['FILE'] == '2014C-3857.filt.vcf') | (df['FILE'] == '2014C-3600.filt.vcf')]
dfs

In [ ]:
variation_service = VariationService(session)

variation_service.insert_variants(dfs, ref_contigs, core_masks)

In [ ]:
s = session.query(Sample).first()
s

In [ ]:
s.sample_sequences[1].get_core_mask().core_proportion()

In [ ]:
variation_service.pairwise_distance(['2014C-3857', '2014C-3600'], var_type='snp')
#variation_service.pairwise_distance(sample_names, var_type='snp')

In [ ]:
v = session.query(VariationAllele).filter_by(position = 16854).all()
v

In [ ]:
v[0].sequence

In [ ]:
r = session.query(ReferenceSequence).filter_by(sequence_name = 'JASV01000003.1').first()
r

In [ ]:
r.variants

In [ ]:
s = session.query(Sample).filter_by(name='2014C-3857').first()
s

In [ ]:
s.sample_sequences

In [ ]:
s.variants[0].id

In [ ]:
v = session.query(VariationAllele).filter_by(id='JASV01000001.1:16854:T:C').first()
v.samples

In [ ]:
ss = session.query(SampleSequence).first()
ss

In [ ]:
ss.get_core_mask().core_length()

In [ ]:
ss.get_core_mask().core_proportion()

In [ ]:
print(ss.sequence)
print(ss.sample)

In [ ]:
len(ss.get_core_mask())

# References

In [ ]:
from biocommons.seqrepo import SeqRepo
import ga4gh.vrs.dataproxy as dataproxy

sr = SeqRepo("data/references")
ref_proxy = dataproxy.SeqRepoDataProxy(sr)
ref_proxy.get_sequence('NCBI:JASV01000002.1', 0, 10)

# Construct alignment

In [ ]:
reference = session.query(Reference).first()
reference

In [ ]:
sequence = reference.sequences[0]
sequence.sequence_name

In [ ]:
sample_sequences = session.query(SampleSequence).filter(SampleSequence.sequence_id == sequence.id).all()
sample_sequences

In [ ]:
sample_sequences[0].sample

In [ ]:
session.query()

In [ ]:
reference.sequences[1].variants

In [ ]:
samples = [session.query(Sample).first()]
samples

In [ ]:
samples[0].variants[0].sequence

In [ ]:
for sample in samples:
    for variant in sample.variants:
        

In [ ]:
samples[0].variants

In [ ]:
import Bio
import pandas as pd
import ga4gh.vrs.dataproxy as dataproxy
import sqlalchemy

class CoreAlignmentConstructor:
    
    def __init__(self, session: sqlalchemy.orm.session.Session, reference_data_proxy: dataproxy.SeqRepoDataProxy):
        self._session = session
        self._ref_proxy = reference_data_proxy
        
    def construct_alignment(self, reference_seq: ReferenceSequence, samples: List[Sample], include_reference: bool = True) -> Bio.Align.MultipleSeqAlignment:
        file_seqs = {}
        chrom_alignments = {}
        all_files = set(variants_df['FILE'])
        for chrom in set(variants_df['CHROM']):
            seq = Bio.Seq.Seq(self._ref_proxy.get_sequence(f'NCBI:{chrom}'))
            df_chrom = variants_df[variants_df['CHROM'] == chrom]
            positions = list(set(df_chrom['POS']))
            positions.sort()
            for pos in positions:
                ref = seq[pos-1:pos]

                # if in core
                if self._core_masks[chrom][pos]:
                    df_pos = df_chrom[df_chrom['POS'] == pos]
                    for file in all_files:
                        if chrom not in file_seqs:
                            file_seqs[chrom] = {}
                        if file not in file_seqs[chrom]:
                            file_seqs[chrom][file] = Bio.Seq.Seq(data='')

                        if file in set(df_pos['FILE']):
                            df_file = df_pos[df_pos['FILE'] == file]
                            alt = str(df_file['ALT'].values[0])
                            ref_file = str(df_file['REF'].values[0])

                            if ref_file != ref:
                                raise Exception(f'Error: reference from VCF [{ref_file}] != reference from genome [{ref}] for {chrom}:{pos}:{file}')

                            file_seqs[chrom][file] += alt
                        else:
                            file_seqs[chrom][file] += ref

                    if include_reference:
                        # Add the reference sequence in
                        if 'reference' not in file_seqs[chrom]:
                            file_seqs[chrom]['reference'] = Bio.Seq.Seq(data='')

                        file_seqs[chrom]['reference'] += ref

        for chrom in file_seqs:
            chrom_records = [Bio.SeqRecord.SeqRecord(file_seqs[chrom][file], id=file) for file in file_seqs[chrom]]
            chrom_alignments[chrom] = Bio.Align.MultipleSeqAlignment(chrom_records)
                
        chroms = sorted(chrom_alignments.keys())
        chr1 = chroms.pop()
        core_snv_align = chrom_alignments[chr1]

        for chrom in chroms:
            core_snv_align += chrom_alignments[chrom]
                
        return core_snv_align
        
align_constructor = CoreAlignmentConstructor(session, ref_proxy)
alignment = align_constructor.construct_alignment(df[df['TYPE'] == 'snp'])
print(alignment)